# Extracting liq pts data, Dropping liq pts with bad data, spatial joining good liq pts data to cells

# Two options: proper way or simply extract at each point and spatial join to cells

# Let's try the proper way first for a moderately sized event (Chiba) and see how long it takes/how accurate it gets

In [1]:
# Alex Chansky
# 3/14/21
# Purpose: Extracting values at each point within liquefaction 1 km buffer, 
import arcpy
from arcpy import env
import pandas as pd
import os
arcpy.CheckOutExtension("Spatial") # This checks out the license
from arcpy.sa import *
# If getting "Not Signed into Portal" message, should open up and log into ArcGIS Pro

In [2]:
arcpy.env.overwriteOutput = True

In [3]:
import datetime
region_path = "C:\\Users\\achans01\\Downloads\\WatBods\\"
path = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\"
temp_path = "C:\\Users\\achans01\\Downloads\\Arc"
eq_list = pd.read_csv("R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\z_eq_names\\2021_eq_names.csv")
num_eq = eq_list.shape[0]
eq_list_df = pd.DataFrame(eq_list)

In [5]:
ct = datetime.datetime.now() 
print("current time:-", ct)
for x in range(35,num_eq):
    eq_name = eq_list['NewFolder'][x]
    region = eq_list['Region'][x]
    # Only doing this for events with liquefaction because NL events do not have small liquefaction buffers
    if eq_list['Working'][x]==1 and eq_list['Liq'][x]==1:

        # Create folder for extracted values to sit
        arcpy.management.CreateFolder(path + eq_name, "LiqSampleExtractions")
        arcpy.management.CreateFolder(path + eq_name, "Output")
        # Copy polygon and place in the new folder
        poly = path + eq_name + "\\fish_liq_poly_count.shp"
        poly_copy = path + eq_name + "\\Output\\fish_liq_poly_count.shp"
        arcpy.CopyFeatures_management(poly, poly_copy)
        #arcpy.DeleteField_management(poly_copy, ["Join_Count","TARGET_FID","Id","Id_1"])
        
        # Define all parameters
        dc = region_path + "ASTER_DCs\\"+ region + "_ocean.tif"
        dr = region_path + "ASTER_DRs\\"+ region + "_river.tif"
        dl = region_path + "ASTER_DLs\\"+ region + "_lake.tif"   
        wbd_elev = region_path + "ASTER_DEMs\\" + region + "_wbd_elev.tif"
        AI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\AI\\ai_et0.tif" 
        CTI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_CTI_clipped.tif"
        elev = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_GMTED_clipped.tif"
        elev_std = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_GMTED_std_clipped.tif"
        precip = "R:\\Liquefaction_RemoteSensing\Projects\\GeospatialModel\\global_data_2020\\Precipitation\\wc2.1_prec_annual.tif"
        slope = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_slope.tif"
        soil_thickness = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\Soils\\Global_Soil_Regolith_Sediment_1304\\data\\average_soil_and_sedimentary-deposit_thickness.tif"
        TPI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_TPI.tif"
        TRI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_TRI.tif" 
        upland_lowland = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\Soils\\Global_Soil_Regolith_Sediment_1304\\data\\land_cover_mask.tif"
        vs30 = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\vs30\\global_vs30.grd"
        wtd = "R:\\Liquefaction_RemoteSensing\\Projects\\LossEstimation\\FindingLSE\\GlobalDatasets\\global_wtd_fil1.tif"
        PGA = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pga_mean.flt"
        PGA_std = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pga_std.flt"
        PGV = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pgv_mean.flt"
        PGV_std = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pgv_std.flt"
        
        points = path + eq_name + "\\fish_liq_point.shp"
        # Let's copy this before we start messing with it. One for each set of extractions
        for i in range(1,16):
            points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(i) + ".shp"
            arcpy.CopyFeatures_management(points, points_copy)

        extractparams(eq_name)
        
        target_features = path + eq_name + "\\Output\\fish_liq_poly_count.shp"
        # Only doing this for events with liquefaction because NL events do not have small liquefaction buffers
        for i in range(1,16):
            join_features = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(i) + ".shp"
            out_feature_class = path + eq_name + "\\Output\\fish_liq_poly_count_" + str(i) + ".shp"
            arcpy.analysis.SpatialJoin(target_features, join_features, out_feature_class, "", "", "", "CLOSEST_GEODESIC","1000 Meters")
            arcpy.DeleteField_management(out_feature_class, 
                                 ["Join_Count","TARGET_FID","Id","POINT_X_1","POINT_Y_1"])
            target_features = out_feature_class 
        for i in range(1,15):
            arcpy.Delete_management(path + eq_name + "\\Output\\fish_liq_poly_count_" + str(i) + ".shp")
            
    print("Successfully extracted liq " + str(x), eq_name)
    ct = datetime.datetime.now() 
    print("current time:-", ct)

current time:- 2021-03-15 10:15:14.248525


SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

SystemError: <built-in function isinstance> returned a result with an error set

Successfully extracted liq 35 Nihonkai
current time:- 2021-03-15 10:39:13.767919
Successfully extracted liq 36 Nisqually
current time:- 2021-03-15 10:44:39.605408
Successfully extracted liq 37 Northridge
current time:- 2021-03-15 10:50:52.665908
Successfully extracted liq 38 Oklahoma
current time:- 2021-03-15 10:55:25.959040
Successfully extracted liq 39 Piedmont
current time:- 2021-03-15 10:55:25.959040
Successfully extracted liq 40 Pisco
current time:- 2021-03-15 10:55:25.959040
Successfully extracted liq 41 PugetSound
current time:- 2021-03-15 11:07:28.959810
Successfully extracted liq 42 PugetSound1949
current time:- 2021-03-15 11:17:59.583121
Successfully extracted liq 43 Samara
current time:- 2021-03-15 11:23:16.011434
Successfully extracted liq 44 SanSimeon
current time:- 2021-03-15 11:28:00.379625
Successfully extracted liq 45 Tecoman
current time:- 2021-03-15 11:32:30.363136
Successfully extracted liq 46 TelireLimon
current time:- 2021-03-15 11:38:17.068617
Successfully extrac

In [4]:
# Extracting each layer (or at least similar) into a shapefile so we can easily discard values of each poor resolution parameter
# while keeping values of high-resolution parameters. After dropping poor resolution parameters, will spatial resolution closest.
def extractparams(eq_name):
    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(1) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[dc,"DC"],[dr,"DR"],[dl,"DL"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(2) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[wbd_elev,"WBD_Elev"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(3) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[AI,"AI"]],"NONE")
    aa = arcpy.SelectLayerByAttribute_management(points_copy, 'NEW_SELECTION', '"AI" = -999')
    arcpy.management.DeleteFeatures(aa)

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(4) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[CTI,"CTI"]],"NONE")
    aa = arcpy.SelectLayerByAttribute_management(points_copy, 'NEW_SELECTION', '"CTI" = -9999')
    arcpy.management.DeleteFeatures(aa)

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(5) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[elev,"Elev"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(6) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[elev_std,"Elev_std"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(7) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[precip,"Precip"]],"NONE")
    aa = arcpy.SelectLayerByAttribute_management(points_copy, 'NEW_SELECTION', '"Precip" = -9999')
    arcpy.management.DeleteFeatures(aa)

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(8) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[slope,"Slope"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(9) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[soil_thickness,"Soil_thick"]],"NONE")
    aa = arcpy.SelectLayerByAttribute_management(points_copy, 'NEW_SELECTION', '"Soil_thick" = -9999')
    arcpy.management.DeleteFeatures(aa)

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(10) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[TPI,"TPI"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(11) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[TRI,"TRI"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(12) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[upland_lowland,"Upland_low"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(13) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[vs30,"Vs30"]],"NONE")

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(14) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[wtd,"WatTabDep"]],"NONE")
    aa = arcpy.SelectLayerByAttribute_management(points_copy, 'NEW_SELECTION', '"WatTabDep" = 0')
    arcpy.management.DeleteFeatures(aa)

    points_copy = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(15) + ".shp"
    ExtractMultiValuesToPoints(points_copy, [[PGA,"PGA"],[PGA_std,"PGA_std"],[PGV,"PGV"],[PGV_std,"PGV_std"]],"NONE")

# Realized I cut out -999 for AI instead of -9999. Possibly -999 was there for some cells as well which would be why I chose that. Now going to try deleting points of -9999 and spatial joining all AI to cells again. May need to drop AI from current cell layers first. Afterwards will also have to extract values to CSV (quick)

In [13]:
# Extraction is already done. Now deleting sampled AI values of -9999.
ct = datetime.datetime.now() 
print("current time:-", ct)
for x in range(11,num_eq):
    eq_name = eq_list['NewFolder'][x]
    region = eq_list['Region'][x]
    # Only doing this for events with liquefaction because NL events do not have small liquefaction buffers
    if eq_list['Working'][x]==1 and eq_list['Liq'][x]==1:

        #arcpy.DeleteField_management(poly_copy, ["Join_Count","TARGET_FID","Id","Id_1"])
        join_features = path + eq_name + "\\LiqSampleExtractions\\fish_liq_point_copy_" + str(3) + ".shp"
        aa = arcpy.SelectLayerByAttribute_management(join_features, 'NEW_SELECTION', '"AI" = -9999')
        arcpy.management.DeleteFeatures(aa)

        # Dropping the pre-existing AI field (skip for Achaia because already done)
        target_features = path + eq_name + "\\Output\\fish_liq_poly_count_" + str(15) + ".shp"
        arcpy.DeleteField_management(target_features, ["AI"])
        
        # Name of new polygon
        out_feature_class = path + eq_name + "\\Output\\fish_liq_poly_count_" + str(16) + ".shp"
        # Adding new AI field
        arcpy.analysis.SpatialJoin(target_features, join_features, out_feature_class, "", "", "", "CLOSEST_GEODESIC","1000 Meters")
        arcpy.DeleteField_management(out_feature_class, 
                                 ["Join_Count","TARGET_FID","Id","POINT_X_1","POINT_Y_1"])

        # Delete one ending in 15 because it has incorrect (skip for Achaia because want to double check first)
        arcpy.Delete_management(path + eq_name + "\\Output\\fish_liq_poly_count_" + str(15) + ".shp")    
        
    print("Successfully extracted liq " + str(x), eq_name)
    ct = datetime.datetime.now() 
    print("current time:-", ct)

current time:- 2021-03-16 15:46:42.883624
Successfully extracted liq 11 Darfield
current time:- 2021-03-16 15:49:41.116536
Successfully extracted liq 12 Denali
current time:- 2021-03-16 15:49:53.149263
Successfully extracted liq 13 Duzce
current time:- 2021-03-16 15:50:03.457510
Successfully extracted liq 14 Emilia
current time:- 2021-03-16 15:50:15.079912
Successfully extracted liq 15 Haiti
current time:- 2021-03-16 15:50:27.397672
Successfully extracted liq 16 HectorMine
current time:- 2021-03-16 15:50:27.397672
Successfully extracted liq 17 Hokkaido
current time:- 2021-03-16 15:50:49.850578
Successfully extracted liq 18 Honduras
current time:- 2021-03-16 15:51:02.404988
Successfully extracted liq 19 Illapel
current time:- 2021-03-16 15:51:15.759556
Successfully extracted liq 20 Iquique
current time:- 2021-03-16 15:51:28.690657
Successfully extracted liq 21 Iwate
current time:- 2021-03-16 15:51:28.690657
Successfully extracted liq 22 Kobe
current time:- 2021-03-16 15:51:59.578894
Suc

In [5]:
eq_name="Achaia"

In [7]:
target_features = path + eq_name + "\\Output\\fish_liq_poly_count_" + str(15) + ".shp"
arcpy.DeleteField_management(target_features, ["AI"])

<Result 'R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\Achaia\\Output\\fish_liq_poly_count_15.shp'>

In [9]:
arcpy.Delete_management(path + eq_name + "\\Output\\fish_liq_poly_count_" + str(15) + ".shp") 

<Result 'true'>